### Connection

In [ ]:
import requests
import sqlite3
import pandas as pd

# URL of the .db file hosted on GitHub
db_url = "https://raw.githubusercontent.com/brutus-the-homeschooler/Capstone/main/Database/acsse_2022.db"

# Download the .db file
response = requests.get(db_url)

# Save the .db file locally
with open("acsse_2022.db", "wb") as db_file:
    db_file.write(response.content)

# Connect to the SQLite database
conn = sqlite3.connect('acsse_2022.db')

# Query the 'census_data' table
census_data_df = pd.read_sql_query("SELECT * FROM census_data", conn)

# Query the 'place_dictionary' table
place_dict_df = pd.read_sql_query("SELECT * FROM place_dictionary", conn)

# Query the 'state_dictionary' table
state_dict_df = pd.read_sql_query("SELECT * FROM state_dictionary", conn)

# Query the 'variable_dictionary' table
variable_dict_df = pd.read_sql_query("SELECT * FROM variable_dictionary", conn)

# Close the connection
conn.close()

# Display data (optional)
#print(census_data_df.head())
#print(place_dict_df.head())
#print(state_dict_df.head())
#print(variable_dict_df.head())


### Simple EDA (complete variables, count places, count state)

In [21]:
nan_counts = census_data_df.isna().sum()

zero_nan_counts = nan_counts[(nan_counts.index.str.endswith('E')) & (nan_counts == 0)]

# Display the count of variables that match the criteria
count_zero_nan_all = zero_nan_counts.count()

unique_states_all = census_data_df['state'].unique()

# Get unique places from the filtered DataFrame
unique_places_all = census_data_df.drop_duplicates(subset=['place', 'state'])

# Display the counts of unique states and unique places
unique_states_count_all = len(unique_states_all)
unique_places_count_all = len(unique_places_all)

print(f"For all populations and states")
print(f"Count of Complete Variables: {count_zero_nan_all}")
print(f"Number of Unique States/Territories: {unique_states_count_all}")
print(f"Number of Unique Places: {unique_places_count_all}")

For all populations and states
Count of Complete Variables: 65
Number of Unique States/Territories: 52
Number of Unique Places: 2399


### City of Dublin EDA

In [ ]:
# Filter the DataFrame based on the specified range for 'K200101_001E'
CoD_df = census_data_df[(census_data_df['state'] == "39") & (census_data_df['place'] == "22694")] # check City of Dublin, OH

# Count NaNs for columns ending in 'E' within the filtered DataFrame
CoD_nan_counts_filtered = CoD_df.isna().sum()

# Filter columns ending with 'E' where NaN count is zero
CoD_zero_nan_counts = CoD_nan_counts_filtered[(CoD_nan_counts_filtered.index.str.endswith('E')) & (CoD_nan_counts_filtered == 0)]

# Display the count of variables that match the criteria
CoD_count_zero_nan = CoD_zero_nan_counts.count()

print(f"Count of Complete Variables: {CoD_count_zero_nan}")

291 complete variables across the data set for City of Dublin. Now we can look to see how many are reliable via the measurement of error and the CV.

In [22]:
# Identify all estimate columns (assuming they end with "E")
estimate_cols = [col for col in CoD_df.columns if col.endswith("E")]

# Create a dictionary to store CV columns
cv_data = {}

# Calculate all CV columns
for est_col in estimate_cols:
    # Find columns that match the MOE pattern
    matching_moe_cols = CoD_df.columns[CoD_df.columns.str.contains(est_col.replace("E", "M"))]

    # Check if any matching MOE column exists
    if matching_moe_cols.empty:
        continue  # Skip to the next estimate column

    moe_col = matching_moe_cols[0]  # Use the first matching MOE column
    cv_col = f'CV_{est_col}'
    cv_data[cv_col] = (CoD_df[moe_col] / CoD_df[est_col]) * 100  # Calculate CV

# Create a new DataFrame with the CV columns
cv_df = pd.DataFrame(cv_data)

# Concatenate the new CV DataFrame with CoD_df
CoD_df = pd.concat([CoD_df, cv_df], axis=1)

In [23]:
# Set your desired CV threshold
cv_threshold = 15

# Identify valid variables where CV < 15%
valid_variables_chunk = [est_col for est_col in estimate_cols if f'CV_{est_col}' in CoD_df.columns and (CoD_df[f'CV_{est_col}'] < cv_threshold).all()]

# Append valid variables to the main list
all_valid_variables = []
all_valid_variables.extend(valid_variables_chunk)

print(f"Count of Variables for City of Dublin with CV threshold less than {cv_threshold}: {len(all_valid_variables)}")

Count of Variables for City of Dublin with CV threshold less than 15: 101
